# Lesson 1: Router Engine

Welcome to Lesson 1.

To access the `requirements.txt` file, the data/pdf file required for this lesson and the `helper` and `utils` modules, please go to the `File` menu and select`Open...`.

I hope you enjoy this course!

In [29]:
!pip uninstall -y llama-index
!pip uninstall -y llama_index
!pip uninstall -y llama-index-core
!pip uninstall -y llama-index-llms-openai
!pip uninstall -y llama-index-embeddings-openai

Found existing installation: llama-index 0.11.19
Uninstalling llama-index-0.11.19:
  Successfully uninstalled llama-index-0.11.19


Found existing installation: llama-index-core 0.11.19
Uninstalling llama-index-core-0.11.19:
  Successfully uninstalled llama-index-core-0.11.19
Found existing installation: llama-index-llms-openai 0.2.16
Uninstalling llama-index-llms-openai-0.2.16:
  Successfully uninstalled llama-index-llms-openai-0.2.16
Found existing installation: llama-index-embeddings-openai 0.2.5
Uninstalling llama-index-embeddings-openai-0.2.5:
  Successfully uninstalled llama-index-embeddings-openai-0.2.5


In [30]:
!pip install llama-index-core
!pip install llama-index-llms-openai
!pip install llama-index-embeddings-openai

  Using cached llama_index_core-0.11.19-py3-none-any.whl.metadata (2.4 kB)
Using cached llama_index_core-0.11.19-py3-none-any.whl (1.6 MB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-agent-openai 0.3.4 requires llama-index-llms-openai<0.3.0,>=0.2.9, which is not installed.
llama-index-cli 0.3.1 requires llama-index-embeddings-openai<0.3.0,>=0.2.0, which is not installed.
llama-index-cli 0.3.1 requires llama-index-llms-openai<0.3.0,>=0.2.0, which is not installed.
llama-index-multi-modal-llms-openai 0.2.3 requires llama-index-llms-openai<0.3.0,>=0.2.11, which is not installed.
llama-index-program-openai 0.2.0 requires llama-index-llms-openai<0.3.0,>=0.2.0, which is not installed.
llama-index-question-gen-openai 0.2.0 requires llama-index-llms-openai<0.3.0,>=0.2.0, which is not installed.


  Using cached llama_index_llms_openai-0.2.16-py3-none-any.whl.metadata (3.3 kB)
Using cached llama_index_llms_openai-0.2.16-py3-none-any.whl (13 kB)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-cli 0.3.1 requires llama-index-embeddings-openai<0.3.0,>=0.2.0, which is not installed.


  Using cached llama_index_embeddings_openai-0.2.5-py3-none-any.whl.metadata (686 bytes)
Using cached llama_index_embeddings_openai-0.2.5-py3-none-any.whl (6.1 kB)


In [1]:
pip install --upgrade llama-index llama-index-llms-openai llama-index-embeddings-openai

  Using cached llama_index-0.11.19-py3-none-any.whl.metadata (11 kB)
Using cached llama_index-0.11.19-py3-none-any.whl (6.8 kB)
Note: you may need to restart the kernel to use updated packages.


## Setup

In [2]:
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [ ]:
pip install --upgrade llama-index

In [14]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

## Load Data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

In [4]:
from llama_index.core import SimpleDirectoryReader

# load documents
documents = SimpleDirectoryReader(input_files=["metagpt.pdf"]).load_data()

c:\Users\kosmo\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


## Define LLM and Embedding model

In [5]:
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

In [7]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.llm = OpenAI(model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

## Define Summary Index and Vector Index over the Same Data

In [8]:
from llama_index.core import SummaryIndex, VectorStoreIndex

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

## Define Query Engines and Set Metadata

In [9]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()

In [11]:
from llama_index.core.tools import QueryEngineTool


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to MetaGPT"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the MetaGPT paper."
    ),
)

## Define Router Query Engine

In [12]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector


query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [15]:
response = query_engine.query("What is the summary of the document?")
print(str(response))

Selecting query engine 0: This choice indicates that the document is useful for summarization questions related to MetaGPT..
The document discusses MetaGPT, a meta-programming framework that utilizes Standardized Operating Procedures (SOPs) to improve multi-agent systems based on Large Language Models (LLMs). It introduces various roles such as Product Manager, Architect, Engineer, Project Manager, and QA Engineer in the software development process. MetaGPT incorporates efficient human workflows, structured communication interfaces, and an executable feedback mechanism to enhance code generation quality. The software development process of a "Drawing App" using MetaGPT is outlined, highlighting the roles of different agents like the Architect, Project Manager, Engineer, and QA Engineer. The software employs Python libraries for GUI creation and color selection. The document also discusses MetaGPT's performance in generating executable code, the significance of clear requirements in so

In [16]:
print(len(response.source_nodes))

34


In [17]:
response = query_engine.query(
    "How do agents share information with other agents?"
)
print(str(response))

Selecting query engine 1: This choice is more relevant as it specifically mentions retrieving specific context, which is necessary for understanding how agents share information with other agents..
Agents share information with other agents by utilizing a shared message pool where they can publish structured messages. This shared message pool allows all agents to exchange messages directly, enabling them to both publish their own messages and access messages from other agents transparently. Additionally, agents can subscribe to relevant messages based on their role profiles, allowing them to extract the information they need for their specific tasks and responsibilities.


## Let's put everything together

In [18]:
from utils import get_router_query_engine

query_engine = get_router_query_engine("metagpt.pdf")

In [19]:
response = query_engine.query("Tell me about the ablation study results?")
print(str(response))

Selecting query engine 1: The ablation study results are specific context from the MetaGPT paper, making choice 2 the most relevant..
The ablation study results show that MetaGPT effectively addresses challenges related to context utilization, code hallucinations, and information overload in the development of recommendation engine software. By accurately interpreting natural language descriptions, maintaining information validity, and focusing on granular tasks like requirement analysis and package selection, MetaGPT mitigates issues such as ambiguity, incomplete implementation, missing dependencies, and undiscovered bugs. Additionally, the use of a global message pool and subscription mechanism helps manage information overload by filtering out irrelevant contexts and streamlining communication for enhanced efficiency.


In [ ]:
!pip list


In [ ]:
!pip show llama-index-core

In [ ]:
!pip show llama-index-core
!pip show llama-index-llms-openai